# Conjugate Gradient Method

## 경사하강법 (Steepest Decent method)

$A$ 행렬이 Positive Definite인 경우 $Ax=b$ 문제는 다음 에너지 $\phi$ 를 최소화 하는 문제와 같다.

$$
\min_x \phi(x) = \frac{1}{2} x^T A x - x^T b
$$

잔차 (Residual) $r(x)$ 은 다음과 같이 정의한다.

$$
r(x) = -\nabla \phi(x) = b - Ax
$$

현 위치에서 가장 변화율이 심한 방향은 $r(x)=-\nabla \phi(x)$ 이다.
경사 하강법은 이 방향으로 해를 찾는 방법이다. 즉

$$
x_{k+1} = x_{k} + \alpha r_{k} \\
r_{k} = r(x_{k})
$$

여기서 $\alpha$ 는 학습률이다. 에너지 식의 Variation이 0이 되는 학습률을 구하면 다음과 같다.

$$
\delta \phi = \phi (x_{k} + \alpha r_{k})-\phi (x_{k}) \\
\frac{d\Delta \phi}{d\alpha} = r_k^T A x_k + \alpha r_k^T A r_k - r_k^T b = r_k^T r_k - \alpha r_k^T A r_k=0
$$

즉

$$
\alpha_k = \frac{r_k^T r_k}{r_k^T A r_k}.
$$

In [8]:
import numpy as np
from cg_gpu import *

In [2]:
def sd(A, b, x0=None, tol=1e-12, itmax=5000, verbose=False):
    """
    Steepest Decent method
        
    Parameters
    ----------
    A : array
        Operation matrix
    b : array
        Forcing term
    tol : float
        tolerance
    itmax : integer
        maximum iteration
    verbose : bool
        print intermediate solution
    
    Result
    ------
    x : array
        solution
    """
    if not x0:
        # Initial guess
        x = b.copy()
    else:
        x = x0

    for n in range(itmax):
        # Compute Residual
        r = b - A @ x
        r2 = r.T @ r

        if np.sqrt(r2) < tol:
            break
        else:
            # Compute Learning rate
            alpha =  r2 / (r.T @ A @ r)

            # Update learning rate
            x += alpha*r
            
        if verbose:
            print(n+1, x)
        
    return x, n+1

## Conjugate Gradient Method
해를 찾는 방향 벡터가 $r_k$ 가 아니고 $d_k$ 라 하자.

$$
x_{k+1} = x_{k} + \alpha d_k
$$

이 경우에도 $\delta \phi=0$ 을 만족시키는 학습률은 아래와 같다.

$$
\alpha_k = \frac{r_k^T d_k}{d_k^T A d_k}.
$$

Conjugate Gradient method는 방향 벡터 $d_k$ 가 $A$ Conjugate (또는 $A$-Orthognal)을 만족한다.

$$
d_i^T A d_j = 0~~~\textrm{for all}~i \neq j.
$$

이 방향벡터 $d_k$ 는 residual $r_k$ 의 선형 결합으로 구성한다.

$$
d_k = r_k + \beta_k d_{k-1}
$$

$A$ Conjugate 를 만족하기 위해서는 위 식에서 양변에 $d_{k-1}^T A$를 곱한다.

$$
d_{k-1}^T A d_k = d_{k-1}^T A (r_k + \beta_k d_{k-1}).
$$

$d_{k-1}^T A d_k=0$ 이므로 $\beta_k$ 는 다음과 같다.

$$
\beta_k = -\frac{d_{k-1}^T A r_k}{d_{k-1}^T A d_{k-1}}
$$

다음 성질을 만족한다.

### Theorem 1 (Orthogonality of residual)
$$
r_{k+1}^T r_i = 0~~\textrm{for all}~i \le k 
$$

#### Proof
$x_k$ 는 $x_0$ 와 $d_1, d_2,...,d_k$ 의 선형 결합으로 표현할 수 있다. 

$$
x_{k+1} = x_0 + D_k y
$$

$D_k$ 를 이들 방향 벡터로 구성한 Matrix이다. 이를 에너지 식에 적용하면

$$
\phi(x_0 + D_k y) = \phi(x_0) + \frac{1}{2} y^T (D_k^T A D_k) y - y^T D_k^T(b - Ax_0)
$$

에너지가 최소화 되어야 하므로

$$
(D_k^T A D_k) y = D_k^T (b - Ax_0).
$$

즉

$$
D_k^T (b- Ax_0) - (D_k^T A D_k) y = D^k (b - A(x_0 + D_ky)) = D_k ^T r_{k+1} = 0
$$

그러므로 

$$
r_{k+1}^T d_i = 0~~\textrm{for all}~i \le k
$$

이를 이용하면 $i \le k$ 인 경우

$$
r_{k+1}^T r_i = r_{k+1}^T (d_i - \beta_i d_{i-1})=0
$$

즉 Residual은 서로 수직임을 알 수 있다.


### 구현
Residual의 직교성을 이용하면
$$
d_k - r_k = \beta_k d_{k-1} \\
$$

이므로

$$
r_k^T(d_k - r_k) = \beta_k r_k^T d_{k-1}=0
$$

그러므로

$$
r_k^T d_k = r_k^T r_k
$$

즉

$$
\alpha_k = \frac{r_k^T d_k}{d_k^T A d_k} = \frac{r_k^T r_k}{d_k^T A d_k}
$$

$r_k=b - Ax_k$ 이므로

$$
r_{k+1} - r_k = -A (x_{k+1} - x_k) = -\alpha_k A d_k.
$$

이를 이용하면

$$
\beta_{k+1} = -\frac{d_{k}^T A r_{k+1}}{d_{k}^T A d_{k}} =  -\frac{r_{k+1}^T(r_{k+1} - r_k)}{d_k^T(r_{k+1} - r_k)}
$$

여기서 $d_k^T r_{k+1}=0$, $r_k^T d_k= r_k^T r_k$ 이고

$$
r_{k+1}^T r_k= r_{k+1}^T (d_k + \beta_k d_{k-1}) = r_{k+1}^T d_k + \beta_k r_{k+1}^Td_{k-1} = 0
$$

을 적용하면

$$
\beta_{k+1} =  \frac{r_{k+1}^Tr_{k+1}}{r_k^T r_k}.
$$


$$
r_k = b - Ax_{k}\\
r_{k} - r_{k-1} = - A (x_{k} - x_{k-1}) = -\alpha_{k-1} A d_{k-1} \\
d_i^T (r_k - r_{k-1}) = 0
$$

### Algorithm

$$
\begin{align}
& \text{Compute} \; \; r_0 := b - Ax_0, \; p_0 := r_0 \\
& \text{For} \; j = 0, 1, \cdots, \; \text{until convergence Do:} \\
& \qquad \alpha_j \,:=\, (r_j, r_j) / (A_{p_j}, p_j) \\
& \qquad x_{j+1} \,:=\, x_j + \alpha_j p_j \\
& \qquad r_{j+1} \,:=\, r_j - \alpha_j A p_j \\
& \qquad \beta_j \,:=\, (r_{j+1}, r_{j+1}) / (r_j, r_j) \\
& \qquad p_{j+1} \,:=\, r_{j+1} + \beta_j p_j \\
& \text{EndDo} \\
\end{align}
$$

In [3]:
def cg(A, b, x0=None, tol=1e-12, itmax=5000, verbose=False):
    """
    Conjugate Gradient Method
        
    Parameters
    ----------
    A : array
        Operation matrix
    b : array
        Forcing term
    tol : float
        tolerance
    itmax : integer
        maximum iteration
    verbose : bool
        print intermediate solution
    
    Result
    ------
    x : array
        solution
    """
    
    if not x0:
        # Initial guess
        x = b.copy()
    else:
        x = x0
        
    r = b - A @ x
    r2 = r @ r

    for n in range(itmax):
        if np.sqrt(r2) < tol:
            break

        else:
            # Direction vector
            if n == 0:
                d = r.copy()    # 이거 조심
            else:
                # Update direction
                beta = r2 / r20  # 이거 r2로 하면 안되는걸까?
                d = r + beta*d
                
            Ad = A @ d

            # Update x and residual
            alpha = r2 / (d @ Ad)
            x += alpha*d
            r -= alpha*Ad
            
            # Update magnitude of residual
            r20 = r2    # previous r
            r2 = r @ r  # new r 이 두개로 Beta구하려고 업데이트
            
        if verbose:
            print(n+1, x)
            
    return x, n+1

In [4]:
# Example
A = np.diag([1,2,3,4])
b = np.array([1,1,1,1])

print('Solution by linear algebra:\n', np.linalg.solve(A, b))
print('Solution by steepest desent:\n', sd(A, b, x0=[0,0,0,0], verbose=True)[0])
print('Solution by congugate gradient:\n', cg(A, b, x0=[0.,0.,0.,0.], verbose=True)[0])

Solution by linear algebra:
 [1.         0.5        0.33333333 0.25      ]
1 [0.4 0.4 0.4 0.4]
2 [0.64 0.48 0.32 0.16]
3 [0.784 0.496 0.336 0.304]
4 [0.8704 0.4992 0.3328 0.2176]
5 [0.92224 0.49984 0.33344 0.26944]
6 [0.953344 0.499968 0.333312 0.238336]
7 [0.9720064 0.4999936 0.3333376 0.2569984]
8 [0.98320384 0.49999872 0.33333248 0.24580096]
9 [0.9899223  0.49999974 0.3333335  0.25251942]
10 [0.99395338 0.49999995 0.3333333  0.24848835]
11 [0.99637203 0.49999999 0.33333334 0.25090699]
12 [0.99782322 0.5        0.33333333 0.2494558 ]
13 [0.99869393 0.5        0.33333333 0.25032652]
14 [0.99921636 0.5        0.33333333 0.24980409]
15 [0.99952982 0.5        0.33333333 0.25011755]
16 [0.99971789 0.5        0.33333333 0.24992947]
17 [0.99983073 0.5        0.33333333 0.25004232]
18 [0.99989844 0.5        0.33333333 0.24997461]
19 [0.99993906 0.5        0.33333333 0.25001523]
20 [0.99996344 0.5        0.33333333 0.24999086]
21 [0.99997806 0.5        0.33333333 0.25000548]
22 [0.99998684 0.

In [5]:
# Finite Difference Matrix
def fd(n):
    A = np.zeros((n,n))

    for i in range(n):
        A[i,i] = 2
        if i > 0:
            A[i-1,i] = -1
        if i < n-1:
            A[i+1, i] = -1
            
    return A

In [11]:
n = 512

A = fd(n)
b = np.random.rand(n)

sol_cpu = cg(A,b)
sol_gpu = cg_gpu(A,b,b,n)

print(np.linalg.norm(sol_cpu[0]-sol_gpu[0])/n**2)

#print('Solution by linear algebra:\n', np.linalg.solve(A, b))
print('Solution by steepest desent:\n', sol_cpu)
print('Solution by congugate gradient:\n', sol_gpu)

1.651202540464739e-07
Solution by steepest desent:
 (array([  129.95964426,   259.56644622,   388.1850707 ,   516.04504989,
         642.98944333,   769.62008644,   895.77378473,  1021.27581903,
        1146.05105734,  1270.03319283,  1393.71929715,  1516.90059832,
        1639.39163252,  1761.47595317,  1883.38835428,  2005.28786701,
        2126.73658285,  2248.17334536,  2369.39427477,  2490.60939686,
        2611.00103907,  2730.87606857,  2850.03421058,  2968.32703995,
        3085.81427767,  3202.42853233,  3318.64501027,  3434.57729823,
        3550.28763289,  3665.59647594,  3780.04888384,  3893.75215676,
        4006.47175327,  4118.22915175,  4229.57873194,  4340.74401763,
        4451.77294473,  4562.07389649,  4672.29028715,  4782.0756406 ,
        4891.6314765 ,  5000.7793663 ,  5109.58244866,  5218.28607289,
        5326.38484086,  5434.15520402,  5541.63365828,  5648.94319473,
        5755.32941887,  5861.07737578,  5966.03258031,  6070.00910445,
        6173.93256682,  

## GPU 병렬화

Conjugate Gradient는 크게 5가지의 연산으로 구성된다.

- SpMV (Sparse matrix-vector product)
- dot product
- scalar-vector product
- vector addition and subtraction

이 연산들은 SIMD 패턴을 가지고 있으므로 GPU에서 계산하기에 적절하다.

따라서 각 연산을 GPU에서 계산하는 커널을 작성하여 계산 시간을 단축할 수 있다.